In [39]:
n = 5
for i in range(n):
    print("*"*n)
    n -=1

*****
****
***
**
*


In [4]:
# i = 0
n = 6
for i in range(n):
    print("*"*i)
    # n -=1
    


*
**
***
****
*****


In [14]:
n = 5
for i in range(n):
    print("*"*n)
    n -=1
    for j in range(n):
     print("*"*j)

*****

*
**
***
****

*
**
***

*
**

*


In [53]:
n = 5
m = 0
for i in range(n):
    m=i
    for j in range(n):
        if m<n:
            print("", end=" ")
        
        else:
            print("*", end=" ")
        
        m+=1

    print(end="\n")
    

     
    * 
   * * 
  * * * 
 * * * * 


In [54]:
n = 5
m = 0
for i in range(n):
    print("*"*n)
    n =1
    for j in range(n):
        if m>n:
            print("*", end="")
        
        else:
            print("*", end="")
        
        m-=1

    print(end="\n")


*****
*
*
*
*
*
*
*
*
*


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.utils as vutils
import os

# Create output directory for generated images
os.makedirs("generated_images", exist_ok=True)
print("✅ Created output directory for generated images.")

# Loss function
criterion = nn.BCELoss()
print("✅ Initialized Binary Cross Entropy Loss.")

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
print("✅ Initialized optimizers.")

# Training parameters
num_epochs = 1
z_dim = 100  # Latent space dimension
print(f"✅ Training will run for {num_epochs} epochs with latent dimension {z_dim}.")

# Training loop
for epoch in range(num_epochs):
    print(f"\n🚀 Starting Epoch {epoch+1}/{num_epochs}...")
    total_loss_D, total_loss_G = 0, 0  # Track losses

    for i, (real_images, _) in enumerate(dataloader):
        batch_size = real_images.size(0)
        real_images = real_images.cuda()
        
        # Print batch processing info
        print(f"🖼️ Processing batch {i+1}/{len(dataloader)}, Batch size: {batch_size}")

        # Train Discriminator
        real_labels = torch.ones(batch_size, 1).cuda()
        fake_labels = torch.zeros(batch_size, 1).cuda()

        optimizer_D.zero_grad()
        outputs = discriminator(real_images).view(-1, 1)
        loss_real = criterion(outputs, real_labels)
        print(f"🔵 Discriminator loss on real images: {loss_real.item():.4f}")

        z = torch.randn(batch_size, z_dim, 1, 1).cuda()
        fake_images = generator(z)
        outputs = discriminator(fake_images.detach()).view(-1, 1)
        loss_fake = criterion(outputs, fake_labels)
        print(f"🔴 Discriminator loss on fake images: {loss_fake.item():.4f}")

        loss_D = loss_real + loss_fake
        loss_D.backward()
        optimizer_D.step()
        total_loss_D += loss_D.item()

        # Train Generator
        optimizer_G.zero_grad()
        outputs = discriminator(fake_images).view(-1, 1)
        loss_G = criterion(outputs, real_labels)

        loss_G.backward()
        optimizer_G.step()
        total_loss_G += loss_G.item()

        # Print intermediate loss values every 50 batches
        if i % 50 == 0:
            print(f"[Epoch {epoch+1}/{num_epochs}] Step {i}/{len(dataloader)} | D Loss: {loss_D.item():.4f} | G Loss: {loss_G.item():.4f}")

    # Print epoch summary
    avg_loss_D = total_loss_D / len(dataloader)
    avg_loss_G = total_loss_G / len(dataloader)
    print(f"\n✅ Epoch [{epoch+1}/{num_epochs}] Completed!")
    print(f"📉 Average D Loss: {avg_loss_D:.4f} | 🎨 Average G Loss: {avg_loss_G:.4f}")

    # Save generated images every 10 epochs
    if (epoch + 1) % 10 == 0:
        with torch.no_grad():
            print("🎨 Generating sample images...")
            z = torch.randn(16, z_dim, 1, 1).cuda()
            fake_images = generator(z).detach().cpu()
            fake_images = (fake_images + 1) / 2  # Rescale to [0,1]

            grid_img = vutils.make_grid(fake_images, nrow=4)
            save_path = f"generated_images/epoch_{epoch+1}.png"
            vutils.save_image(grid_img, save_path)

            print(f"🖼️ Saved sample images: {save_path}")

print("\n🎉 Training complete! Check 'generated_images' folder for results.")
